In [ ]:
!pip install lxml 

In [ ]:
!mkdir data

In [ ]:
cd data 

/content/data


In [ ]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/face.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/hat.npy

--2022-02-08 16:50:42--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/face.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.128, 142.251.8.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126746224 (121M) [application/octet-stream]
Saving to: ‘face.npy’

face.npy            100%[===================>] 120.87M  27.5MB/s    in 4.4s    

2022-02-08 16:50:47 (27.5 MB/s) - ‘face.npy’ saved [126746224/126746224]

--2022-02-08 16:50:48--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/hat.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174526320 (166M) [application/octet-stream]
Savi

In [ ]:
!ls 

face.npy  hat.npy


In [ ]:
cd ..

/content


In [ ]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

In [ ]:
# define some constants
N_OBJE = 2
OBJE = {0: "Face", 1: "Hat"}

# number of samples to take in each class
N = 5000

# some other constants
N_EPOCHS = 10


files = ["face.npy", "hat.npy"] 

In [ ]:
def load(dir, reshaped, files):
    "Load .npy or .npz files from disk and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    "Takes a list or a list of lists and returns its normalized form"

    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    "Takes a list or a list of lists and returns its denormalized form"

    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [ ]:
#This line of code is for the trial purpose only. 

obje = load("data/", False, ['face.npy'])  

In [ ]:
#This line of code is for the trial purpose only. 

visualize(obje[0][0])

In [ ]:
#second argument is True for reshaping the image to a 28x28 form. A conv net expects this format.
obje = load("data/", True, files)
 
#second argument is False because we don't need to reshape the image. An MLP net expects this format.
#obje = load("data/", False, files)


# limit no of samples in each class to N
obje = set_limit(obje, N)

# normalize the values
obje = list(map(normalize, obje))

# define the labels
labels = make_labels(N_OBJE, N)

# prepare the data
x_train, x_test, y_train, y_test = tts(obje, labels, test_size=0.05)

# one hot encoding
Y_train = np_utils.to_categorical(y_train, N_OBJE)
Y_test = np_utils.to_categorical(y_test, N_OBJE)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_OBJE, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)

print ("Training complete")

print ("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print (("Accuracy: "), ((score + 0.0) / len(preds)) * 100)


model.save("obje"+ ".h5")
print ("Model saved")

Epoch 1/10
297/297 [==============================] - 13s 9ms/step - loss: 0.1279 - accuracy: 0.9521
Epoch 2/10
297/297 [==============================] - 3s 9ms/step - loss: 0.0689 - accuracy: 0.9757
Epoch 3/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0550 - accuracy: 0.9816
Epoch 4/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0432 - accuracy: 0.9847
Epoch 5/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0325 - accuracy: 0.9891
Epoch 6/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0248 - accuracy: 0.9916
Epoch 7/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0177 - accuracy: 0.9939
Epoch 8/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0185 - accuracy: 0.9931
Epoch 9/10
297/297 [==============================] - 3s 8ms/step - loss: 0.0146 - accuracy: 0.9951
Epoch 10/10
297/297 [==============================] - 2s 8ms/step - loss: 0.0129 - accuracy: 0.995

In [ ]:
!pip install tensorflowjs 

     |████████████████████████████████| 77 kB 2.9 MB/s 


In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras obje.h5 model/

In [ ]:
!zip -r model.zip model 

  adding: model/ (stored 0%)
  adding: model/model.json (deflated 78%)
  adding: model/group1-shard2of2.bin (deflated 7%)
  adding: model/group1-shard1of2.bin (deflated 7%)


In [ ]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>